# See top co-occurring terms in blog descriptions that mention fandoms

In [7]:
# Load blog descriptions
import pandas as pd

fandom_blog_descriptions_path = '/data/tumblr_community_identity/fandom_blog_descriptions_processed.csv'
blog_info = pd.read_csv(fandom_blog_descriptions_path, sep='\t')
fandoms = blog_info['fandom'].unique()
print(len(fandoms))
blog_info

/usr0/home/mamille2/anaconda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (10,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


33


,tumblelog_id,activity_time_epoch,tumblr_blog_name,tumblr_blog_title,tumblr_blog_description,tumblr_blog_url,is_group_blog,is_primary,is_private,created_time_epoch,...,timezone,language,blog_classifier,generated_date,tumblog_id,post_count,fandom,fandom_term,processed_tumblr_blog_description,tokenized_blog_description
0,123758,1538018782,coldfashion,AP ROCKS YOUR FACE,annelise pruitt i designed the emmy-winning st...,http://coldfashion.tumblr.com/,False,False,False,NaN,...,NaN,NaN,safe,20181021,123758,25,star_wars,star wars,annelise pruitt i designed the emmy-winning st...,annelise pruitt i designed the emmy-winning st...
1,139093,1558154235,hauntedfalcon,Haunted Jaeger,fuck Reylo,https://hauntedfalcon.tumblr.com/,False,False,False,1.196747e+12,...,US/Eastern,en_US,safe,20190518,139093,982,star_wars,reylo,fuck Reylo,fuck reylo
2,340785,1546861232,anomalies,party.,"<p>27 years old <br>i like dogs, pokemon, taki...",http://anomalies.tumblr.com/,False,False,False,1.216068e+12,...,US/Pacific,en_US,safe,20190201,340785,2,pokemon,pokemon,"27 years old i like dogs, pokemon, taking pict...","27 years old i like dogs , pokemon , taking pi..."
3,493522,1556894408,hazal,Smitten hedgehogs like tea,"<br>Hazal. <br> 29 years old, vegetarian, web...",https://hazal.tumblr.com/,False,False,False,1.225645e+12,...,Europe/Istanbul,en_US,safe,20190619,493522,10,supernatural,spn,"Hazal. 29 years old, vegetarian, web designer,...","hazal . 29 years old , vegetarian , web design..."
4,794605,1563544355,sundayriver81,Sundayriver's world,"Italian girl, obsessed with The walking Dead a...",https://sundayriver81.tumblr.com/,False,False,False,1.236731e+12,...,Europe/Amsterdam,en_US,safe,20190719,794605,999,walking_dead,the walking dead,"Italian girl, obsessed with The walking Dead a...","italian girl , obsessed with the walking dead ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319111,444485579,1541257544,bhluetina,lilyy,<p>potterhead.</p><p>the marauders’</p>,https://bhluetina.tumblr.com/,False,False,False,1.541255e+12,...,US/Eastern,en_US,safe,20181201,444485579,4,harrypotter,potterhead,potterhead. the marauders’,potterhead . the marauders ’
319112,444669851,1541965412,zzellak,Zellak,30/ He/Him/ Them/They. 18+ art. NO MINORS OR ...,https://zzellak.tumblr.com/,False,False,False,1.541347e+12,...,US/Eastern,en_US,adult,20181114,444669851,17,undertale,undertale,30/ He/Him/ Them/They. 18+ art. NO MINORS OR ...,30/ he/him/ them/they . 18+ art . no minors or...
319113,444881744,1567866289,peaches-and-nicotine,♡,"ED TW! || 18 || UK || Not Pro || 5'6"" sw: 218l...",https://peaches-and-nicotine.tumblr.com/,False,False,False,1.541449e+12,...,US/Eastern,en_US,safe,20190908,444881744,88,teen_wolf,tw,"ED TW! || 18 || UK || Not Pro || 5'6"" sw: 218l...",ed tw ! || 18 || uk || not pro || 5 ' 6 '' sw ...
319114,446026777,1542862673,littlegleekhunter,Pretty Gleek Hunter❤,Pretty Little Liars/Supernatural & Glee❤😍,https://littlegleekhunter.tumblr.com/,False,False,False,1.542070e+12,...,US/Eastern,en_US,safe,20190616,446026777,4,supernatural,supernatural,Pretty Little Liars/Supernatural & Glee❤😍,pretty little liars/supernatural & glee❤😍


In [8]:
import nltk
import string

stops = nltk.corpus.stopwords.words('english')
stops = [w for w in stops if not w in ['he', 'him', 'she', 'her', 'they', 'them']]
stops += [c for c in string.punctuation]
stops += ['--', "'m", "'s", 'also', "n't", '...', '`', '•', '||', "''", "``", '’', '//', "'re", "'ll"]
stops

# For each fandom, make a counter of terms in blog descriptions
from collections import Counter
fandom_terms = blog_info.groupby('fandom').agg({'tokenized_blog_description': lambda x: ' '.join(x.tolist())})

fandom_words = fandom_terms['tokenized_blog_description'].to_dict()

fandom_word_counters = {}
for fandom, words in fandom_words.items():
    fandom_word_counters[fandom] = Counter([w for w in words.split() if not w in stops])
    
# len(fandom_word_counters)

from pprint import pprint
for fandom, ctr in fandom_word_counters.items():
    print(fandom)
    pprint(ctr.most_common(30))
    print()

academia
[('bnha', 5734),
 ('hero', 2450),
 ('blog', 2274),
 ('academia', 2238),
 ('art', 1534),
 ('like', 1099),
 ('love', 1031),
 ('boku', 953),
 ('anime', 895),
 ('mostly', 848),
 ('currently', 842),
 ('stuff', 841),
 ('she/her', 683),
 ('oc', 642),
 ('things', 624),
 ('fandoms', 581),
 ('free', 573),
 ('post', 568),
 ('please', 527),
 ('draw', 488),
 ('indie', 473),
 ('hi', 457),
 ('ask', 440),
 ('voltron', 435),
 ('one', 432),
 ('trash', 418),
 ('haikyuu', 410),
 ('sometimes', 404),
 ('current', 404),
 ('artist', 403)]

ajin
[('ajin', 75),
 ('blog', 29),
 ('x', 22),
 ('anime', 16),
 ('hunter', 14),
 ('tokyo', 14),
 ('art', 14),
 ('free', 13),
 ('♥', 12),
 ('ghoul', 12),
 ('fandoms', 9),
 ('currently', 9),
 ('page', 8),
 ('mostly', 8),
 ('ga', 8),
 ('love', 8),
 ('academia', 8),
 ('stuff', 8),
 ('v', 7),
 ('lot', 7),
 ('one', 7),
 ('black', 7),
 ('hero', 7),
 ('la', 6),
 ('de', 6),
 ('el', 6),
 ('days', 6),
 ('man', 6),
 ('mp100', 6),
 ('manga', 6)]

allmarvel
[('marvel', 29348),
 

[('pokemon', 19903),
 ('blog', 6346),
 ('like', 4757),
 ('love', 3470),
 ('things', 3417),
 ('stuff', 2982),
 ('art', 2774),
 ('anime', 2459),
 ('games', 2277),
 ('name', 1910),
 ('post', 1904),
 ('ask', 1526),
 ('mostly', 1502),
 ('video', 1445),
 ('hi', 1429),
 ('free', 1379),
 ('anything', 1348),
 ('time', 1225),
 ('music', 1175),
 ('please', 1169),
 ('feel', 1105),
 ('life', 1078),
 ('old', 1075),
 ('reblog', 1067),
 ('hello', 1060),
 ('enjoy', 1049),
 ('welcome', 1045),
 ('want', 1039),
 ('lot', 1037),
 ('fan', 1018)]

power rangers
[('power', 478),
 ('rangers', 464),
 ('·', 118),
 ('blog', 92),
 ('love', 89),
 ('sentai', 88),
 ('super', 65),
 ('like', 63),
 ('things', 55),
 ('stuff', 51),
 ('fan', 47),
 ('anime', 46),
 ('games', 43),
 ('comics', 35),
 ('rider', 34),
 ('video', 33),
 ('kamen', 32),
 ('music', 30),
 ('find', 30),
 ('name', 29),
 ('anything', 29),
 ('mostly', 28),
 ('2017', 28),
 ('gay', 28),
 ('old', 28),
 ('ranger', 28),
 ('time', 26),
 ('mighty', 25),
 ('movies',

In [12]:
# Search for ages
import re
pattern = r'^\d+$'

for fandom, ctr in fandom_word_counters.items():
    print(fandom)
#     matches = [item for item in ctr.most_common() if re.search(pattern, item[0]) is not None and \
#             int(item[0]) < 85 and int(item[0]) > 14 and \
#             item[1] >= 5]
#     for word, count in matches:
#         print(f'{word}\t{count}')
    matches = {word: ctr[word] for word in ctr if re.search(pattern, word) is not None and \
            int(word) <= 80 and int(word) >= 15}
    for i in range(15,81):
#         print(f'{i}\t{matches.get(str(i), 0)}')
        print(matches.get(str(i), 0))
    print()

academia
83
109
128
192
247
224
202
156
120
93
55
47
46
20
11
19
4
2
0
2
0
1
0
0
0
2
1
0
0
1
0
0
1
0
1
9
1
0
0
0
0
0
0
0
0
3
1
0
2
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
5

ajin
0
2
0
1
4
3
1
3
2
0
0
0
1
0
0
0
1
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0

allmarvel
230
365
526
886
829
958
904
711
619
497
399
288
224
177
121
197
66
45
27
26
21
13
11
3
3
20
2
9
3
3
2
1
1
3
1
54
1
9
1
0
0
1
2
0
1
26
0
0
15
0
6
0
1
0
1
19
1
1
1
0
10
2
0
0
2
68

arthurian
12
37
45
65
79
89
83
69
49
53
38
30
21
12
4
20
3
2
2
1
2
2
0
0
0
5
3
2
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
1
0
2
0
0
1
0
0
0
0
0
0
7
0
0
0
0
0
0
0
0
0
4

attack_on_titan
30
45
43
78
78
76
68
47
47
24
18
13
10
6
10
14
2
0
3
0
1
1
0
0
0
2
0
0
0
0
0
0
0
1
2
7
0
0
0
0
0
0
0
0
0
5
0
0
1
0
1
0
0
0
0
4
0
0
0
0
0
0
0
0
0
10

dcu
34
61
72
162
147
188
192
170
155
123
130
97
62
34
23
42
13
7
10
3
9
10
6
2
3
5
6
5
1
1
6
1
1
1
1
15
0
30
0
0
1
1
0
0
0
10
0
0
5
1
0
2
1
1
1
8
0
1
1
0
4
0
0
2
0
28

dr_who
15
27
36


# Tokenize, lowercase blog descriptions (use NLTK)

In [1]:
# Load fandom blog descriptions
import pandas as pd

fandom_blog_descriptions_path = '/data/tumblr_community_identity/fandom_blog_descriptions_processed.csv'
blog_info = pd.read_csv(fandom_blog_descriptions_path, sep='\t')
# blog_info = pd.read_csv(fandom_blog_descriptions_path, sep='\t', escapechar='\\', error_bad_lines=False)
blog_info

/usr0/home/mamille2/anaconda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (7,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,tumblelog_id,activity_time_epoch,tumblr_blog_name,tumblr_blog_title,tumblr_blog_description,tumblr_blog_url,is_group_blog,is_primary,is_private,created_time_epoch,updated_time_epoch,timezone,language,blog_classifier,generated_date,tumblog_id,post_count,fandom,fandom_term,processed_tumblr_blog_description
0,123758,1538018782,coldfashion,AP ROCKS YOUR FACE,annelise pruitt i designed the emmy-winning st...,http://coldfashion.tumblr.com/,False,False,False,NaN,NaN,NaN,NaN,safe,20181021,123758,25,star_wars,star wars,annelise pruitt i designed the emmy-winning st...
1,139093,1558154235,hauntedfalcon,Haunted Jaeger,fuck Reylo,https://hauntedfalcon.tumblr.com/,False,False,False,1.196747e+12,1.5463e+12,US/Eastern,en_US,safe,20190518,139093,982,star_wars,reylo,fuck Reylo
2,340785,1546861232,anomalies,party.,"<p>27 years old <br>i like dogs, pokemon, taki...",http://anomalies.tumblr.com/,False,False,False,1.216068e+12,1.54556e+12,US/Pacific,en_US,safe,20190201,340785,2,pokemon,pokemon,"27 years old i like dogs, pokemon, taking pict..."
3,493522,1556894408,hazal,Smitten hedgehogs like tea,"<br>Hazal. <br> 29 years old, vegetarian, web...",https://hazal.tumblr.com/,False,False,False,1.225645e+12,1.54274e+12,Europe/Istanbul,en_US,safe,20190619,493522,10,supernatural,spn,"Hazal. 29 years old, vegetarian, web designer,..."
4,794605,1563544355,sundayriver81,Sundayriver's world,"Italian girl, obsessed with The walking Dead a...",https://sundayriver81.tumblr.com/,False,False,False,1.236731e+12,1.54502e+12,Europe/Amsterdam,en_US,safe,20190719,794605,999,walking_dead,the walking dead,"Italian girl, obsessed with The walking Dead a..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319131,444485579,1541257544,bhluetina,lilyy,<p>potterhead.</p><p>the marauders’</p>,https://bhluetina.tumblr.com/,False,False,False,1.541255e+12,1.54126e+12,US/Eastern,en_US,safe,20181201,444485579,4,harrypotter,potterhead,potterhead. the marauders’
319132,444669851,1541965412,zzellak,Zellak,30/ He/Him/ Them/They. 18+ art. NO MINORS OR ...,https://zzellak.tumblr.com/,False,False,False,1.541347e+12,1.54161e+12,US/Eastern,en_US,adult,20181114,444669851,17,undertale,undertale,30/ He/Him/ Them/They. 18+ art. NO MINORS OR ...
319133,444881744,1567866289,peaches-and-nicotine,♡,"ED TW! || 18 || UK || Not Pro || 5'6"" sw: 218l...",https://peaches-and-nicotine.tumblr.com/,False,False,False,1.541449e+12,1.56527e+12,US/Eastern,en_US,safe,20190908,444881744,88,teen_wolf,tw,"ED TW! || 18 || UK || Not Pro || 5'6"" sw: 218l..."
319134,446026777,1542862673,littlegleekhunter,Pretty Gleek Hunter❤,Pretty Little Liars/Supernatural & Glee❤😍,https://littlegleekhunter.tumblr.com/,False,False,False,1.542070e+12,1.54207e+12,US/Eastern,en_US,safe,20190616,446026777,4,supernatural,supernatural,Pretty Little Liars/Supernatural & Glee❤😍


In [2]:
fandoms = blog_info['fandom'].unique()
print(len(fandoms))
fandoms

42


array(['star_wars', 'pokemon', 'supernatural', 'walking_dead',
       'harrypotter', 'song_ice_fire', 'sherlock', 'tolkien', 'naruto',
       'teen_wolf', 'riverdale', 'dragonage', 'voltron', 'academia',
       'hamilton', 'allmarvel', 'star_trek', 'overwatch',
       'once_upon_a_time', 'haikyuu', 'dr_who', 'undertale', 'homestuck',
       'dcu', 'supercorp', 'arthurian', 'attack_on_titan', 'korra',
       'power rangers', 'stargate', 'ajin', 'world_ends_with_you',
       'youjo senki', 'harry potter', 'marvel', 'hp', 'lord of the rings',
       'reylo', 'rey', 'potterhead', 'klance', 'mcu'], dtype=object)

In [3]:
# Remove error rows with mismatched fandoms
true_fandoms = [
    'ajin',
    'academia',
    'arthurian',
    'dcu',
    'dr_who',
    'dragonage',
    'haikyuu',
    'hamilton',
    'harrypotter',
    'homestuck',
    'allmarvel',
    'naruto',
    'once_upon_a_time',
    'overwatch',
    'pokemon',
    'power rangers',
    'song_ice_fire',
    'riverdale',
    'sherlock',
    'attack_on_titan',
    'star_trek', 
    'star_wars',
    'stargate',
    'supercorp',
    'supernatural',
    'teen_wolf',
    'korra',
    'walking_dead',
    'world_ends_with_you',
    'tolkien',
    'undertale',
    'voltron',
    'youjo senki',
]
print(len(true_fandoms))
blog_info = blog_info[blog_info['fandom'].isin(true_fandoms)]
len(blog_info)

33


319120

In [4]:
# Tokenize blog descriptions (nltk)
import nltk
from tqdm import tqdm_notebook as tqdm

def tokenize_nltk(text):
    return ' '.join([tok.lower() for tok in nltk.word_tokenize(str(text))])

blog_info['tokenized_blog_description'] = [tokenize_nltk(d) for d in tqdm(blog_info['processed_tumblr_blog_description'].tolist())]

/usr0/home/mamille2/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
blog_info.drop_duplicates(subset=['tumblog_id'], inplace=True)
len(blog_info)

/usr0/home/mamille2/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


319116

In [6]:
blog_info.to_csv(fandom_blog_descriptions_path, sep='\t', index=False)

# Old

In [53]:
# Tokenize blog descriptions (spaCy)
import spacy

nlp = spacy.load('en', disable=['tagger', 'parser', 'ner'])
nlp.add_pipe(nlp.create_pipe('sentencizer'))
def tokenize_spacy(text):
    full_text = ''
    full_text = ''
    if len(str(text)) > 100000:
        print('Description too long')
        return full_text
    doc = nlp(str(text))
    for sent in doc.sents:
        full_text += ' '.join([tok.text.lower() for tok in sent]) + ' '
    return full_text.strip()

In [51]:
# Tokenize blog descriptions (stanza)
import stanza

nlp = stanza.Pipeline(lang='en', processors='tokenize')
# done_texts = []
def tokenize_stanza(text):
    full_text = ''
    doc = nlp(str(text))
    for sent in doc.sentences:
        full_text += ' '.join([tok.text for tok in sent.tokens]) + ' '
    return full_text.strip()

2020-11-12 11:40:03 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |

2020-11-12 11:40:03 INFO: Use device: cpu
2020-11-12 11:40:03 INFO: Loading: tokenize
2020-11-12 11:40:03 INFO: Done loading processors!
